In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt


In [ ]:
dir_SLAND  = '/Trendy/Data/SLAND_Trendy-v10_S2_LatLon/'
dir_ctrs   = '/Trendy/Data/data_ancillary/info_countries/'
dir_grids  = '/Trendy/Data/grids/'
dir_forest = '/GCB2021_commentary/Data/forest_masks/'
dir_LSM    = '/Trendy/Data/LSMs_Trendy-v8/'
dir_tmp    = '/Trendy/Data/tmp/'
dir_out    = '/Trendy/Data/SLAND_Trendy-v10_S2_countries/'


## Regrid countries, land-sea mask, and forest area to forest mask

In [ ]:
#Regrid country ISO code to model grid
file_grid      = dir_grids + 'grid_xy_360x720-ForestMask'
fname_ctrs     = dir_ctrs + 'wrld_cntrs_BLUE_TN_upd.nc'
fname_ctrs_reg = dir_tmp + 'wrld_cntrs_BLUE_TN_upd_on_ForestMask_grid.nc'
if os.path.exists(fname_ctrs_reg):  os.remove(fname_ctrs_reg)
os.system('cdo remaplaf,' + file_grid + ' ' + fname_ctrs + ' ' + fname_ctrs_reg)

#Regrid country ISO code to model grid
file_grid = dir_grids + 'grid_xy_360x720-ForestMask'
fname_in  = dir_LSM + 'LSM_reference.nc'
fname_out = dir_LSM + 'LandSeaMask_360x720-ForestMask.nc'
if os.path.exists(fname_out):  os.remove(fname_out)
os.system('cdo remapcon,' + file_grid + ' ' + fname_in + ' ' + fname_out)

#Interpolate forest fraction to grid of forest mask
fname_grid          = dir_grids + 'grid_xy_360x720-ForestMask'
fname_for_frac      = dir_forest + 'ForestFraction_0.5deg_2013.nc'
fname_for_frac_regr = dir_forest + 'ForestFraction_0.5deg_2013_regridded-ForestMask.nc'
if os.path.exists(fname_for_frac_regr):  os.remove(fname_for_frac_regr)
os.system('cdo remapcon,' + fname_grid + " " + fname_for_frac + " " + fname_for_frac_regr)


## Calculate SLAND in forests (using different definitions of forests, and different thresholds for forest cover)

In [ ]:
#Define models
models = ['CABLE-POP', 'CLASSIC', 'CLASSIC-N', 'CLM5.0', 'DLEM', 'IBIS', 'ISAM', 'ISBA-CTRIP', 'JSBACH', 'JULES-ES-1.1',
          'LPJ-GUESS', 'LPJwsl', 'LPX-Bern', 'OCN', 'ORCHIDEE', 'ORCHIDEEv3', 'SDGVM', 'VISIT', 'YIBs']

#Define data sets that should be used
data_sets = ['DGVMs-NaturalLand-PFTs']#, 'DGVMs-Forests-PFTs', 'BLUE']

#Define grid size
gridsize = '360x720-ForestMask'

#Select year for forst mask (2000, 2013, or 2016)
year = '2013'

#Select all forest or non-intact forest
selections = ['non-intact-forests']#, 'intact-forests', 'all-forests']

#Read ISO codes for countries, IPCC countries, and conversions between ISO alpha-3 codes from IPCC and ISO numeric
fname_ctrs_ISO   = dir_tmp + 'wrld_cntrs_BLUE_TN_upd_on_ForestMask_grid.nc'
fname_IPCC_codes = dir_ctrs + 'IPCC_regions.xlsx'
fname_ISO_num    = dir_ctrs + 'iso_codes_alpha_numeric.xlsx'
data_ctrs_ISO   = xr.open_dataset(fname_ctrs_ISO)
data_IPCC_codes = pd.read_excel(fname_IPCC_codes, sheet_name='region_classification', header=0, usecols=[0, 1, 3])
data_alph_num   = pd.read_excel(fname_ISO_num, header=0)

#Read forest mask
fname_mask   = dir_forest + 'Hansen2010_IFL_' + year + '.nc'
mask_Potapov = xr.open_dataset(fname_mask)

#Read forest fraction in 2013
fname_Hansen = dir_forest + 'ForestFraction_0.5deg_2013_regridded-ForestMask.nc'
data_Hansen  = xr.open_dataset(fname_Hansen)

for data_set in data_sets:
    
    #Define different forest cover thresholds according to Hansen et al. (2013)
    if data_set=='DGVMs-Forests-PFTs':         Hansen_treshs = [0.05, 0.10, 0.15, 0.20, 0.25]#, 0.30, 0.35]
    elif data_set=='DGVMs-NaturalLand-PFTs':   Hansen_treshs = [0.15]    
    
    #Define file name
    if data_set=='BLUE':                      fname_weight = dir_forest + 'Weights_ForestFraction_BLUE_regrid360x720-ForestMask.nc'
    elif data_set=='DGVMs-Forests-PFTs':      fname_weight = dir_forest + 'Weights_ForestFraction_DGVMs-S2-S3_regrid360x720-ForestMask.nc'
    elif data_set=='DGVMs-NaturalLand-PFTs':  fname_weight = dir_forest + 'Weights_NaturalLandCoverFraction_DGVMs-S2-S3_regrid360x720-ForestMask.nc'

    #Read weight factors for forest
    for_weight = xr.open_dataset(fname_weight)

    #Select maximum weighting factor of 50
    if 'NaturalLand' in data_set:
        for_weight = for_weight.w_NLCF
    else:
        for_weight = for_weight.w_FF
    
    #Re-index data
    check_lat1 = np.max(np.abs(data_Hansen['lat'].values - mask_Potapov.lat.values))
    check_lon1 = np.max(np.abs(data_Hansen['lon'].values - mask_Potapov.lon.values))
    check_lat2 = np.max(np.abs(for_weight['lat'].values - mask_Potapov.lat.values))
    check_lon2 = np.max(np.abs(for_weight['lon'].values - mask_Potapov.lon.values))
    if check_lat1>0.001 or check_lon1>0.001:  sys.exit('Latitudes do not agree')
    if check_lat2>0.001 or check_lon2>0.001:  sys.exit('Latitudes do not agree')
    data_Hansen = data_Hansen.reindex({'lat': mask_Potapov['lat'], 'lon': mask_Potapov['lon']}, method='nearest')
    for_weight  = for_weight.reindex({'lat': mask_Potapov['lat'], 'lon': mask_Potapov['lon']}, method='nearest')

    #Loop over different forest cover thresholds according to Hansen et al. (2013)
    for Han2013_thresh in Hansen_treshs:
    
        #Loop over different forest definitions
        for selection in selections:

            #Select which forests to include (see Grassi et al. (2021), who used 20% threshold for forest cover)
            if selection=='non-intact-forests':
                mask_forest = data_Hansen.forest_fraction.where(mask_Potapov.Band1!=2)
                mask_forest = mask_forest > 0.0
            elif selection=='intact-forests':
                mask_forest = data_Hansen.forest_fraction.where(mask_Potapov.Band1==2)
                mask_forest = mask_forest > 0.0
            elif selection=='all-forests':
                mask_forest = data_Hansen.forest_fraction > 0.0

            #Create dicts for storing data
            SLAND_ctrs = dict()

            #Define output file name
            fname_out = dir_out + 'SLAND-S2-countries_' + selection + '_Weights-' + data_set + '_MaxForCover' + '{:.2f}'.format(Han2013_thresh) + '_vRemapToForestMask-Hansen2010_IFL_' + year + '.xlsx'
            if os.path.exists(fname_out): os.remove(fname_out)

            #Create xlsx-file (it will be filled at end of loop with country data from every model)
            with pd.ExcelWriter(fname_out) as writer:

                #Loop over models
                for model in models:

                    print(model)

                    #Get file name for SLAND
                    fnames = [file for file in os.listdir(dir_SLAND) if (model + '_' in file) and (gridsize in file) and ('SLAND_NBP' in file) and ('NBPPFT' not in file)]
                    if len(fnames)!=1:  sys.exit('Filename not unique')

                    #Read SLAND data
                    fname = dir_SLAND + fnames[0]
                    data_SLAND = xr.open_dataset(fname)

                    #Get lat and lon names
                    if 'latitude' in data_SLAND.dims:  lat_name, lon_name = 'latitude', 'longitude'
                    else:                              lat_name, lon_name = 'lat', 'lon'

                    #Check that model grid and country grid agree
                    check_lat1 = np.max(np.abs(data_SLAND[lat_name].values - mask_forest.lat.values))
                    check_lon1 = np.max(np.abs(data_SLAND[lon_name].values - mask_forest.lon.values))
                    check_lat2 = np.max(np.abs(data_ctrs_ISO.lat.values - mask_forest.lat.values))
                    check_lon2 = np.max(np.abs(data_ctrs_ISO.lon.values - mask_forest.lon.values))
                    if check_lat1>0.01:  sys.exit('Latitudes do not agree')
                    if check_lon1>0.01:  sys.exit('Longitudes do not agree')
                    if check_lat2>0.01:  sys.exit('Latitudes do not agree')
                    if check_lon2>0.01:  sys.exit('Longitudes do not agree')            

                    #Re-index if there are small deviations in lat and lon
                    if (check_lat1!=0) or (check_lon1!=0):  data_SLAND    = data_SLAND.reindex({lat_name: mask_forest['lat'], lon_name: mask_forest['lon']}, method='nearest')
                    if (check_lat2!=0) or (check_lon2!=0):  data_ctrs_ISO = data_ctrs_ISO.reindex({lat_name: mask_forest['lat'], lon_name: mask_forest['lon']}, method='nearest')

                    #Get weights (if no weights for specific model exist, use the average of all other models)
                    if data_set=='BLUE':
                        weight = for_weight
                    else:
                        if model in for_weight.model:
                            weight = for_weight.sel(model=model)
                        else:
                            print('Averaging data from other models for ' + model)
                            weight = for_weight.mean('model')

                    #Apply forest cover thershold
                    weight = weight.where(data_Hansen.forest_fraction>Han2013_thresh)                       
                        
                    #Apply forest mask
                    data_SLAND = (data_SLAND * weight).where(mask_forest==1)

                    #Loop over all country codes
                    for i, iso_alpha3 in enumerate(data_IPCC_codes['ISO']):

                        if np.mod(i, 50)==0:
                            print('  -run ' + str(i+1) + ' of ' + str(len(data_IPCC_codes['ISO'])))

                        #Get numbeic ISO code of country
                        iso_numeric = data_alph_num['Numeric'][data_alph_num['Alpha-3 code']==iso_alpha3].values[0]

                        #Select country in country mask
                        mask_ISO = data_ctrs_ISO.ISOcode==iso_numeric

                        #Get SLAND sum in selected country
                        data_sel = data_SLAND.where(mask_ISO).sum(('lat', 'lon'))

                        #Convert to Tg C/year
                        data_sel = 1000 * data_sel

                        #Save values in dict
                        SLAND_ctrs[iso_alpha3] = data_sel.SLAND.values

                    #Special cases for certain IPCC countries
                    SLAND_ctrs['SXM'] = SLAND_ctrs['MAF']                                          # Saint Martin is French part of island with Sint Maarten (Dutch part) -> same values are counted for both
                    SLAND_ctrs['ANT'] = SLAND_ctrs['BES'] + SLAND_ctrs['CUW'] + SLAND_ctrs['SXM']  # Netherlands Antilles (Bonaire, Saint Eustatius & Saba + Curacao + Sint Maarten)

                    #Convert data to data frame (and sort by country name)
                    SLAND_ctrs_df = pd.DataFrame(SLAND_ctrs, index=data_SLAND.time.dt.year)
                    SLAND_ctrs_df = SLAND_ctrs_df.reindex(sorted(SLAND_ctrs_df.columns), axis=1)

                    #Adde units in first cell
                    SLAND_ctrs_df = SLAND_ctrs_df.rename_axis('unit: TG C/year')

                    #Create sheet in xlsx for every model and store country data
                    SLAND_ctrs_df.to_excel(writer, sheet_name=model + '_SLAND_IPCC_ctrs', index=True, header=True, float_format='%.6f')

#Remove regridded country mask and regridded forest fraction
if os.path.exists(fname_ctrs_reg):  os.remove(fname_ctrs_reg)
if os.path.exists(fname_Hansen):    os.remove(fname_Hansen)


## Calculate total weighted SLAND

In [ ]:
#Define models
models = ['CABLE-POP', 'CLASSIC', 'CLASSIC-N', 'CLM5.0', 'DLEM', 'IBIS', 'ISAM', 'ISBA-CTRIP', 'JSBACH', 'JULES-ES-1.1',
          'LPJ-GUESS', 'LPJwsl', 'LPX-Bern', 'OCN', 'ORCHIDEE', 'ORCHIDEEv3', 'SDGVM', 'VISIT', 'YIBs']

#Define grid size
gridsize = '360x720-ForestMask'

#Select year for forst mask (2000, 2013, or 2016)
year = '2013'

#Read ISO codes for countries, IPCC countries, and conversions between ISO alpha-3 codes from IPCC and ISO numeric
fname_ctrs_ISO   = dir_tmp + 'wrld_cntrs_BLUE_TN_upd_on_ForestMask_grid.nc'
fname_IPCC_codes = dir_ctrs + 'IPCC_regions.xlsx'
fname_ISO_num    = dir_ctrs + 'iso_codes_alpha_numeric.xlsx'
data_ctrs_ISO   = xr.open_dataset(fname_ctrs_ISO)
data_IPCC_codes = pd.read_excel(fname_IPCC_codes, sheet_name='region_classification', header=0, usecols=[0, 1, 3])
data_alph_num   = pd.read_excel(fname_ISO_num, header=0)

#Read weight factors for forest
fname_weight = dir_forest + 'Weights_ForestFraction_DGVMs-S2-S3_regrid360x720-ForestMask.nc'
for_weight = xr.open_dataset(fname_weight)
for_weight = for_weight.w_FF

#Re-index data if necessary
check_lat2 = np.max(np.abs(for_weight['lat'].values - data_ctrs_ISO.lat.values))
check_lon2 = np.max(np.abs(for_weight['lon'].values - data_ctrs_ISO.lon.values))
if check_lat1>0.001 or check_lon1>0.001:  sys.exit('Latitudes do not agree')
if check_lat2>0.001 or check_lon2>0.001:  sys.exit('Latitudes do not agree')
for_weight  = for_weight.reindex({'lat': data_ctrs_ISO['lat'], 'lon': data_ctrs_ISO['lon']}, method='nearest')

#Create dicts for storing data
SLAND_ctrs = dict()

#Define output file name
fname_out = dir_out + 'SLAND-S2-countries_Weights-' + data_set + '_total-SLAND_vRemapToForestMask-Hansen2010_IFL_' + year + '.xlsx'
if os.path.exists(fname_out): os.remove(fname_out)

#Create xlsx-file (it will be filled at end of loop with country data from every model)
with pd.ExcelWriter(fname_out) as writer:

    #Loop over models
    for model in models:

        print(model)

        #Get file name for SLAND
        fnames = [file for file in os.listdir(dir_SLAND) if (model + '_' in file) and (gridsize in file) and ('SLAND_NBP' in file) and ('NBPPFT' not in file)]
        if len(fnames)!=1:  sys.exit('Filename not unique')

        #Read SLAND data
        fname = dir_SLAND + fnames[0]
        data_SLAND = xr.open_dataset(fname)

        #Get lat and lon names
        if 'latitude' in data_SLAND.dims:  lat_name, lon_name = 'latitude', 'longitude'
        else:                              lat_name, lon_name = 'lat', 'lon'

        #Get weights (if no weights for specific model exist, use the average of all other models)
        if model in for_weight.model:
            weight = for_weight.sel(model=model)
        else:
            weight = for_weight.mean('model')

        #Define weight for all land and apply weighting
        weight = weight.where(weight>0, 1)
        data_SLAND = data_SLAND * weight

        #Loop over all country codes
        for i, iso_alpha3 in enumerate(data_IPCC_codes['ISO']):

            if np.mod(i, 50)==0:
                print('  -run ' + str(i+1) + ' of ' + str(len(data_IPCC_codes['ISO'])))

            #Get numbeic ISO code of country
            iso_numeric = data_alph_num['Numeric'][data_alph_num['Alpha-3 code']==iso_alpha3].values[0]

            #Select country in country mask
            mask_ISO = data_ctrs_ISO.ISOcode==iso_numeric

            #Get SLAND sum in selected country
            data_sel = data_SLAND.where(mask_ISO).sum(('lat', 'lon'))

            #Convert to Tg C/year
            data_sel = 1000 * data_sel

            #Save values in dict
            SLAND_ctrs[iso_alpha3] = data_sel.SLAND.values

        #Special cases for certain IPCC countries
        SLAND_ctrs['SXM'] = SLAND_ctrs['MAF']                                          # Saint Martin is French part of island with Sint Maarten (Dutch part) -> same values are counted for both
        SLAND_ctrs['ANT'] = SLAND_ctrs['BES'] + SLAND_ctrs['CUW'] + SLAND_ctrs['SXM']  # Netherlands Antilles (Bonaire, Saint Eustatius & Saba + Curacao + Sint Maarten)

        #Convert data to data frame (and sort by country name)
        SLAND_ctrs_df = pd.DataFrame(SLAND_ctrs, index=data_SLAND.time.dt.year)
        SLAND_ctrs_df = SLAND_ctrs_df.reindex(sorted(SLAND_ctrs_df.columns), axis=1)

        #Adde units in first cell
        SLAND_ctrs_df = SLAND_ctrs_df.rename_axis('unit: TG C/year')

        #Create sheet in xlsx for every model and store country data
        SLAND_ctrs_df.to_excel(writer, sheet_name=model + '_SLAND_IPCC_ctrs', index=True, header=True, float_format='%.6f')

#Remove regridded country mask and regridded forest fraction
if os.path.exists(fname_ctrs_reg):  os.remove(fname_ctrs_reg)
if os.path.exists(fname_Hansen):    os.remove(fname_Hansen)
